# Tasks of each team member

### David Liu
- Wrote the following functions
  - getHTMLString(url): create a method to parse HTML from the URL
  - json_output(json_lst): compile all the outputs into a json
- Documented the usage of the functions written
- Split and assigned programming tasks for teammates
- Project Description
- Installation Guide
- Structured Data (JSON) Usage

### Sicheng Jiang
 
Get all faculty URLs from the faculty directory page

input: Home Page URL ("https://cs.illinois.edu/about/people/all-faculty")

output: list of String url (example: ["https://cs.illinois.edu/about/people/all-faculty/zaher", "https://cs.illinois.edu/about/people/all-faculty/vadve"]



### Zhaokuan Chen
Then get all the faculty names, phone, email, title, faculty URL

output: (example: [{name: "abc", phone:"123", email:"", title:"professor", facultyURL:"abc"}, {name: "abc", phone:"123", email:"", title:"professor", facultyURL:"abc"}]


# Project Description
In this project, we will use python BeautifulSoup package to parse information about UIUC faculty information, then store this information into a JSON file as structured data for future use. 

# Installation Guide 


*   Use Google Colab without the need to install or setup the environment, or
*   Install the following packages by python package manager and run the code in Jupyter Notebook
 *   bs4
 *   html5lib
 *   requests
 *   spacy
 *   urllib
 





In [ ]:
'''
Import Python libraries installed by Python package manager
'''
from bs4 import BeautifulSoup
import html5lib
import requests
import json
import spacy
import re
from urllib.parse import urlparse

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
'''
faculty_directory_url is the link to query for all the links to faculties
'''
faculty_directory_url = 'https://cs.illinois.edu/about/people/all-faculty'

David's Part 1

In [14]:
def getHTMLString(url):
  """
  parse HTML from the URL
  Input: string of an URL
    Type: String
    Example: "https://cs.illinois.edu/about/people/all-faculty"
  Output: HTML string, request object
    Type: String, Object
  """
  r = requests.get(url)
  s = r.content.decode()
  return s, r

# print(getHTMLString("https://www.geeksforgeeks.org/data-structures/"))

Sicheng's Part

In [ ]:
base_url = 'https://' + urlparse(faculty_directory_url).netloc
nlp = spacy.load('en_core_web_sm')

In [ ]:
faculty_url_pattern = re.compile(r'.*(faculty|people).*', re.IGNORECASE)
def is_faculty_tag(tag):
  """
  Determine if an HTML tag contains a link to a faculty page
  Input: a BeautifulSoup Tag object
  Output: whether input tag contains a link to a faculty page
    Type: Bool
  """
  if tag.name == 'a' and tag.has_attr('href'):
    href = tag['href']
    m = faculty_url_pattern.match(href)
    if m:
      pos_tagged = nlp(tag.get_text())
      for e in pos_tagged.ents:
        if e.label_ == 'PERSON':
          return True
  return False

In [ ]:
def find_faculty_pages(faculty_directory_url):
  """
  Find all links to faculty pages in a faculty directory page.
  Input: an url to a faculty directory page.
    Type: String
    Example: "https://cs.illinois.edu/about/people/all-faculty"
  Output: a list of urls of faculty pages.
    Type: List[String]
  """
  content, res = getHTMLString(faculty_directory_url)
  if not res.ok:
    print("Cannot access Faculty Directory! Error Code: ", res.status_code)
  soup = BeautifulSoup(content, 'html5lib')
  tags = soup.find_all(is_faculty_tag)
  results = [base_url + tag['href'] for tag in tags]
  return results


In [ ]:
faculty_urls = find_faculty_pages(faculty_directory_url)

Zhaokuan's part

In [ ]:
def find_faculty_info(faculty_urls):
    """
    Scrape information from professors' webpage including 
    name, title, phone number, email, office address, professers' homepage URL,
    education history.
    Input: list of URLs of professors webpage
    Type: List
    Output: list of JSON that contains the scraped information
    """
    output = []
    for url in faculty_urls:
        r = requests.get(url)
        #professor name  
        soup = BeautifulSoup(r.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install html5lib
        page_header = soup.find('div', id='hero', class_='container-fluid')
        professor_name = page_header.find('h1').text.strip()
        # print(professor_name)

        #professor title
        page_content = soup.find('div', id='content_inner', class_='container-fluid')
        roles = page_content.find('div', class_='roles')
        title = roles.find('div', class_='title').text.strip()
        # print(title)

        #professor phone number
        phone_number = roles.find('div', class_='phone').text.strip()
        # print(phone_number)

        #professor email
        email = roles.find('div', class_='email').text.strip()
        # print(email)

        #professor office
        office_address = roles.find('div', class_='office').text.strip()
        # print(office_address)

        #professor URL
        url_section = page_content.find('ul')
        professor_link = [i['href'].strip() for i in url_section.find_all('a', href=True)] if url_section else None
        # print(professor_link)

        #professor Education
        education_section = page_content.find_all('ul')[1] if len(page_content.find_all('ul'))>=2 else None
        education = [i.text.strip() for i in education_section.find_all('li')] if education_section else None
        # print(education)

        information = {"name": professor_name, "title": title, "phone_number": phone_number, "email": email, "office_address": office_address, "link": professor_link, "education": education}
        # print(information)
        output.append(information)
    return output

In [ ]:
json_lst = find_faculty_info(faculty_urls)

In [ ]:
# Print out the number of faculty parsed
print(len(json_lst))

143


David's Part 2

In [ ]:
def json_output(json_lst):
  """
  compile all the outputs into a JSON file 'faculty_info.json'
  Input: list of JSON
    Type: List
  Output: None
  """
  with open('faculty_info.json', 'w') as outfile:
    json.dump(json_lst, outfile)
  return 

In [ ]:
json_output(json_lst)

# Structured Data (JSON) Usage
The output JSON file was intended to be used as structured data for future usage. 

### Example


```
{
        "name": "Sarita V Adve",
        "title": "Richard T. Cheng Professor",
        "phone_number": "(217) 333-8461",
        "email": "sadve@illinois.edu",
        "office_address": "4104 Siebel Center for Comp Sci",
        "link": [
            "http://www.cs.uiuc.edu/~sadve"
        ],
        "education": [
            "Computer Science, Ph.D., University of Wisconsin-Madison, 1993"
        ]
    }
```



It contains the following field:
- name: Name of the faculty
- title: Title of the faculty
- phone_number: Phone number of the faculty
- email: Email of the faculty
- office_address: Office address of the faculty
- link: Link of the faculty
- education: Education of the faculty

In [13]:
# Print the JSON output
print(json_lst)

[{'name': 'Sarita V Adve', 'title': 'Richard T. Cheng Professor', 'phone_number': '(217) 333-8461', 'email': 'sadve@illinois.edu', 'office_address': '4104 Siebel Center for Comp Sci', 'link': ['http://www.cs.uiuc.edu/~sadve'], 'education': ['Computer Science, Ph.D., University of Wisconsin-Madison, 1993']}, {'name': 'Vikram Adve', 'title': 'Donald B. Gillies Professor in Computer Science', 'phone_number': '(217) 244-2016', 'email': 'vadve@illinois.edu', 'office_address': '4235 Siebel Center for Comp Sci', 'link': ['http://vikram.cs.illinois.edu/', 'http://llvm.org'], 'education': ['Ph.D. in Computer Science from University of Wisconsin-Madison, 1993.']}, {'name': 'Gul A Agha', 'title': 'Research Professor and Professor Emeritus', 'phone_number': '(217) 244-3087', 'email': 'agha@illinois.edu', 'office_address': '2104 Siebel Center for Comp Sci', 'link': ['http://osl.cs.illinois.edu/members/agha.html'], 'education': ['Ph.D., Computer & Communication Science, University of Michigan at Ann